If the Video is not from the youtube or there is already downloaed videos in the formta of mp4, we have to extract the content from the video from scratch

downside of this method:

- text extraction may be inaccurate in spelling, amounts ...ect
- some text extraction process may not able to pick the text from the audios
- Sentences spererations could be inaccurate

In [6]:
import speech_recognition as sr 
# import moviepy.editor as mp
import pandas as pd
import numpy as np
import glob
import os

In [33]:
list_of_videos = glob.glob('Client_Video_Files/*.mp4')
list_of_ppts =  glob.glob('../../Data/text_univ_chichago/*.pptx')


## Start with some Video Exploratory analysis

In [34]:
# get video files size on the video in Bytes

def file_size(vid_file):
    import os
    byts = os.path.getsize(vid_file)
    # then we return the bytes into bytes, metabyte, gb
    return byts, np.round(byts/1000000, 3), np.round(byts/1000000000, 3)

video_size = [file_size(i) for i in list_of_videos]
video_size

[(255680222, 255.68, 0.256),
 (49122115, 49.122, 0.049),
 (29499314, 29.499, 0.029),
 (137598834, 137.599, 0.138),
 (21366835, 21.367, 0.021)]

In [35]:
# get video files durations 
def video_duration(vid_file):
    from moviepy.editor import VideoFileClip
    clip = VideoFileClip(vid_file)
    return clip.duration,  np.round(clip.duration/60, 3)

video_duration = [video_duration(i) for i in list_of_videos]
video_duration

[(184.36, 3.073),
 (102.06, 1.701),
 (201.51, 3.358),
 (225.66, 3.761),
 (52.25, 0.871)]

## Step 1: convert Video into Audio files 

In [39]:
# Function to read all the video clips and convert to audio files 
def convert_to_audio(vd,  output_pth):
    clip = mp.VideoFileClip(vd) 
    ad_nm = vd.split("/")[-1].split(".")[0]
    clip.audio.write_audiofile(output_pth + "/" + ad_nm + ".wav")
    print("Audio {} has been successfully extracted in {}".format(ad_nm, output_pth))

In [40]:
# convert all video clips into Audio files 
for vid in list_of_videos:
    convert_to_audio(vd = vid, output_pth = 'Audios')

MoviePy - Writing audio in Audios/birds_intro_flamingo_v.wav


MoviePy - Done.
Audio birds_intro_flamingo_v has been successfully extracted in Audios
MoviePy - Writing audio in Audios/birds_intro_peacock_v.wav


MoviePy - Done.
Audio birds_intro_peacock_v has been successfully extracted in Audios
MoviePy - Writing audio in Audios/birds_intro_tailor_bird_v.wav


MoviePy - Done.
Audio birds_intro_tailor_bird_v has been successfully extracted in Audios
MoviePy - Writing audio in Audios/insects_intro_ant_v.wav


MoviePy - Done.
Audio insects_intro_ant_v has been successfully extracted in Audios
MoviePy - Writing audio in Audios/insects_intro_beetle_v.wav


MoviePy - Done.
Audio insects_intro_beetle_v has been successfully extracted in Audios


In [41]:
# audio files size 
list_of_audios = glob.glob('Audios/*.wav')
audio_size = [file_size(i) for i in list_of_audios]
# audio_size

In [42]:
audio_size

[(39806502, 39.807, 0.04),
 (35546442, 35.546, 0.036),
 (18003462, 18.003, 0.018),
 (9216978, 9.217, 0.009),
 (32521182, 32.521, 0.033)]

## Step 2: Convert Audios ino Text files 

In [43]:
# Reading Large Audio Files
# If you want to perform speech recognition of a long audio file, then the below function handles that quite well:
# https://www.thepythoncode.com/article/using-speech-recognition-to-convert-speech-to-text-python

# read all the audo files from the path
# Convert audio into text files 

r = sr.Recognizer()
audio = sr.AudioFile("converted.wav")

# importing libraries 
import speech_recognition as sr 
import os 
from pydub import AudioSegment 
from pydub.silence import split_on_silence 

# create a speech recognition object
r = sr.Recognizer()

# a function that splits the audio file into chunks
# and applies speech recognition
def get_large_audio_transcription(path):
    """
    Splitting the large audio file into chunks
    and apply speech recognition on each of these chunks
    """
    # open the audio file using pydub
    sound = AudioSegment.from_wav(path)  
    # split audio sound where silence is 700 miliseconds or more and get chunks
    chunks = split_on_silence(sound,
        # experiment with this value for your target audio file
        min_silence_len = 500,
        # adjust this per requirement
        silence_thresh = sound.dBFS-14,
        # keep the silence for 1 second, adjustable as well
        keep_silence=500,
    )
    
    folder_name = "audio-chunks"
    # create a directory to store the audio chunks
    if not os.path.isdir(folder_name):
        os.mkdir(folder_name)
    whole_text = ""
    # process each chunk 
    for i, audio_chunk in enumerate(chunks, start=1):
        # export audio chunk and save it in
        # the `folder_name` directory.
        chunk_filename = os.path.join(folder_name, f"chunk{i}.wav")
        audio_chunk.export(chunk_filename, format="wav")
        # recognize the chunk
        with sr.AudioFile(chunk_filename) as source:
            audio_listened = r.record(source)
            # try converting it to text
            try:
                text = r.recognize_google(audio_listened)
            except sr.UnknownValueError as e:
                print("Error:", str(e))
            else:
                text = f"{text.capitalize()}. "
                print(chunk_filename, ":", text)
                whole_text += text
    fl_nm = path.split("/")[-1].split(".")[0]
    res_df = pd.DataFrame({"video_file" : [str(fl_nm)] ,  "raw_txt": [str(whole_text)] } )
    # return the text for all chunks detected and a dataframe that contain the text chunk
    return whole_text, res_df

# convert the convertsed txt into .txt files 
def txt_file(raw_txt,  text_output):
    with open(text_output, mode ='w') as file: 
        file.write("Recognized Speech:") 
        file.write("\n") 
        file.write(raw_txt) 
        print("text ready!")
    

In [44]:
import time
audio_rawtxt = pd.DataFrame()
for audio in list_of_audios:
    print(audio + ": \n")
    extracted_txt = get_large_audio_transcription(path = audio)
    txt_file(raw_txt = extracted_txt[0], text_output = "Audio_Text/" + audio.split("/")[-1].replace(".wav", ".txt") )
    temp = extracted_txt[1]
    audio_rawtxt = pd.concat([audio_rawtxt, temp], axis = 0)
    time.sleep(1)

Audios/insects_intro_ant_v.wav: 

Error: 
Error: 
audio-chunks/chunk3.wav : Did you find an ant hill. 
audio-chunks/chunk4.wav : The app that you see outside or only a small number compared to all the ants below the soil. 
audio-chunks/chunk5.wav : These huge groups of ants are called calm he's a single colony may contain thousands even millions of ants every ant in a colony has a specific job worker answer small ants that are responsible for everything from digging and repairing the tunnels to gathering food to caring for the young ants. 
audio-chunks/chunk6.wav : All of the worker ants all females. 
Error: 
audio-chunks/chunk8.wav : The meal ask job is the fertilized the queen they are larger than the worker ass but not as large as a queen. 
audio-chunks/chunk9.wav : The largest and most important as of the colony is the queen. 
audio-chunks/chunk10.wav : Her job is so late x there are four stages in an ant's life cycle. 
audio-chunks/chunk11.wav : The queen litter eggs after a few d

audio-chunks/chunk15.wav : Can reach heights of 59 in or 150 cm. 
audio-chunks/chunk16.wav : But only way as much as eight pounds are 4 kg. 
audio-chunks/chunk17.wav : The wingspan of a greater flamingo can reach up to 65 inch or 165 cm. 
audio-chunks/chunk18.wav : Flamingos are filter feeders. 
audio-chunks/chunk19.wav : To eat. 
audio-chunks/chunk20.wav : They stand in shallow water and hang their heads down into it. 
audio-chunks/chunk21.wav : They sweep their heads from side-to-side with their bill below the surface. 
audio-chunks/chunk22.wav : And use their spiny town to help them strain food from the water. 
audio-chunks/chunk23.wav : Flamingos eat brine shrimp. 
audio-chunks/chunk24.wav : Algae. 
audio-chunks/chunk25.wav : Insects. 
audio-chunks/chunk26.wav : Mollusks and small crustaceans. 
audio-chunks/chunk27.wav : Because their diet includes both plants and animals. 
audio-chunks/chunk28.wav : Flamingos for omnivores you may be interested to know that flamingos are not born 

In [48]:
audio_rawtxt_copy

,video_file,raw_txt
0,insects_intro_ant_v,Did you find an ant hill. The app that you see...
0,birds_intro_tailor_bird_v,This is a girl named masha. She's sleeping and...
0,birds_intro_peacock_v,Wow look at those feathers. The bird you found...
0,insects_intro_beetle_v,Beetles have a hard shell like armor. And i sa...
0,birds_intro_flamingo_v,Let's learn about flamingos flamingos are tall...


In [51]:
# read all the txt files and format whole chunk of them into dataframe with video file name as unique identifier
audio_rawtxt_copy = audio_rawtxt[(audio_rawtxt['raw_txt'] != "")& ( audio_rawtxt['raw_txt'].str.len() > 5)]


In [53]:
audio_rawtxt_copy.to_csv("scraping_data/Targeted_Ouput/0_raw_audio_rawtxt.csv")

## Step 3 Annotate audio files with timestamp

https://towardsdatascience.com/speech-recognition-with-timestamps-934ede4234b2

the way that work:

1. split each audio in each 5-10 second wav.files while keep track of the second durations in each files incrementally
2. read each audio file and extract the text 
3. and mark each text extracted from each splitted audio file with the incremntally tracked time stamps

In [59]:
# inspared by https://stackoverflow.com/questions/37999150/how-to-split-a-wav-file-into-multiple-wav-files

from pydub import AudioSegment
import math
import re

# function to split audios into different samller clips 
def split_audios_clips(folder, filename, clip_length_sec = 5 ):

    audio = AudioSegment.from_wav(folder + filename)

    # devide the auido by each 5 secs ==> 46 clips
    total_secs = math.ceil(audio.duration_seconds / clip_length_sec)

    sec_per_split = 1 # increment by 1
    start_time = 0
    start_time_lst = []
    end_time_lst = []

    for i in range(0, total_secs, sec_per_split):

        split_fn = str(i) + '_' + filename

        from_sec = i
        to_sec = i+sec_per_split
        t1 = from_sec * clip_length_sec * 1000
        t2 = to_sec * clip_length_sec * 1000

        split_audio = audio[t1:t2]
        split_audio.export(folder + '/splitted_audio/' + split_fn, format="wav")

        start_time_lst.append(start_time)
        start_time = start_time + clip_length_sec
        end_time_lst.append(start_time)
        
    return start_time_lst, end_time_lst

# function to split audio file into smaller files and then extract sentences from each small clips and uniformaly generate time stamp
def generate_timestamped_sentences(audio_path, temp_outpt_wac = 'Audios/splitted_audio/', clip_length_sec = 10 ):
    
    folder = audio_path.split("/")[0]  + "/"
    filename = audio_path.split("/")[1]  
    filetype = audio_path.split("/")[1].split(".")[0]
    
    # divide the audio into clips by 10 seconda each 
    time_stamps = split_audios_clips(clip_length_sec = clip_length_sec, folder = folder, filename = filename)
    
    # save start and end sec time
    time_stamps_df = pd.DataFrame()
    time_stamps_df["start sec"] = time_stamps[0]
    time_stamps_df["end sec"] = time_stamps[1]
    
    splited_audios = glob.glob(temp_outpt_wac + "*wav")
    
    # apply the text extraction in each splitted audio files 
    audio_sentences_timestamp = pd.DataFrame()
    for audio_clip_path in splited_audios:
        if filetype in audio_clip_path:
            extracted_txt = get_large_audio_transcription(path = audio_clip_path )
            temp = extracted_txt[1]
        else:
            pass
    #     audio_sentences_timestamp
        audio_sentences_timestamp = pd.concat([audio_sentences_timestamp, temp], axis = 0)
        
    # cleaned and wrnagel data intopandas df
    audio_sentences_timestamp["index"] = audio_sentences_timestamp.video_file.map(lambda x: int(x.split('_')[0]) )
    audio_sentences_timestamp = audio_sentences_timestamp.set_index("index")
    audio_sentences_timestamp = audio_sentences_timestamp.sort_index(ascending=True)
    
    
    # join with the time_stamps_df
    audio_sentences_timestamp = audio_sentences_timestamp.merge(time_stamps_df, left_index=True, right_index=True)
    audio_sentences_timestamp["audio_source"] = audio_sentences_timestamp.video_file.map(lambda x: re.sub(r'[0-9]', '', re.sub(r'[0-9]_', '', x)) )
    
    # clean divided audio clips from the folder
    for filePath in splited_audios:
        if os.path.exists(filePath):
            os.remove(filePath)
        else:
            print("Can not delete the file as it doesn't exists")
    
    return audio_sentences_timestamp                                                         




In [60]:
audio_word_timestamp_df = pd.DataFrame()

for audio_path in list_of_audios:
    temp_awt = generate_timestamped_sentences(audio_path = audio_path, temp_outpt_wac = 'Audios/splitted_audio/', clip_length_sec= 10 )
    audio_word_timestamp_df = pd.concat([audio_word_timestamp_df, temp_awt], axis = 0)

audio_word_timestamp_df = audio_word_timestamp_df.reset_index(drop=True)

audio-chunks/chunk1.wav : Go to aldi's. 
audio-chunks/chunk1.wav : Locker ants have found a good source of food. 
audio-chunks/chunk2.wav : The reason behind creating asset trail leading back to the colony. 
audio-chunks/chunk1.wav : What's communicate by releasing a scent called family what's the. 
audio-chunks/chunk1.wav : There are four stages in an ant's life cycle. 
audio-chunks/chunk2.wav : The queen litter eggs after feud. 
audio-chunks/chunk1.wav : The abdomen is where an ant stomach is stalwart ants have six legs and. 
audio-chunks/chunk1.wav : Set a responsible for everything from digging and repairing the tunnels to gathering food to caring for the young ants. 
audio-chunks/chunk2.wav : All of the worker ants. 
Error: 
audio-chunks/chunk1.wav : The eggs hatch into love you. 
audio-chunks/chunk2.wav : Each were vast ancestor cocoon around itself. 
audio-chunks/chunk3.wav : Inside the cocoon the larva grows into a. 
Error: 
audio-chunks/chunk1.wav : Even millions of ants. 
aud

audio-chunks/chunk3.wav : Flamingos are also good for. 
audio-chunks/chunk1.wav : Called colonies. 
Error: 
audio-chunks/chunk3.wav : If you watch a flamingo for a little while you may notice it standing on one leg. 
audio-chunks/chunk4.wav : This is not because. 
audio-chunks/chunk1.wav : You may be interested to know that flamingos are not born pink. 
audio-chunks/chunk2.wav : When they are chicks they're actually gray. 
audio-chunks/chunk1.wav : Or 165cm flamingos are filter feeders to eat. 
audio-chunks/chunk1.wav : Bears. 
audio-chunks/chunk2.wav : There are six different species of flamingos but they all have some things in common. 
Error: 
audio-chunks/chunk1.wav : For fight. 
audio-chunks/chunk2.wav : It takes a young flamingo 2 or 3 years before they attain their pink color. 
audio-chunks/chunk3.wav : They acquire their colored. 
audio-chunks/chunk1.wav : They stand in shallow water and hang their heads down into it. 
audio-chunks/chunk2.wav : They sweep their heads from side-

In [61]:
# save this file into csv 
audio_word_timestamp_df.to_csv("scraping_data/Targeted_Ouput/0_raw_audio_word_timestamp.csv", index = False)

## Step 4 Convert meta data ppt into text files 

https://stackoverflow.com/questions/13559133/how-to-open-ppt-file-using-python

https://python-pptx.readthedocs.io/en/latest/api/shapes.html#shape-objects-autoshapes



In [62]:
# function to extract all text from the ppt slides
def read_ppt_text(ppt_path):
    from pptx import Presentation
    prs = Presentation(ppt_path)

    # text_runs will be populated with a list of strings,
    # one for each text run in presentation
    text_runs = []

    for slide in prs.slides:
        for shape in slide.shapes:
    #         print(shape)
            if not shape.has_text_frame:
                continue
            for paragraph in shape.text_frame.paragraphs:
                for run in paragraph.runs:
                    text_runs.append(run.text)
    return text_runs

In [63]:

ppts_names = [i.split("/")[-1].split(".")[0] for i in list_of_ppts]
meta_data_lst = [read_ppt_text(ppt_path = i) for i in list_of_ppts]

metadata_dict = dict(zip(ppts_names, meta_data_lst))

In [64]:
raw_metadata = pd.DataFrame(metadata_dict.items(), columns = ['meta_file', "raw_text"])
raw_metadata.head()

,meta_file,raw_text
0,birds_intro_peafowl_t,"[PEAFOWL, Peacock is the National bird of Indi..."
1,birds_intro_flamingo_t,"[Flamingo, Flamingo is about 110 to 150 cm tal..."
2,insects_intro_ant_t,[Ants are one of the most common insects that ...
3,birds_intro_tailor_bird_t,"[Tailorbird, Tailorbird- The Tailorbird has ta..."
4,insects_intro_beetle_t,[We see many insects around us. Some have wing...


In [66]:
raw_metadata.to_csv("scraping_data/Targeted_Ouput/0_raw_metadata.csv", index = False)